In [94]:
import pandas as pd
import re
import os

file_name = "GMT20200420-190619_Sarah--Cat.transcript notepad.txt"
extension = os.path.splitext(file_name)[1]
base_name = os.path.splitext(file_name)[0]

if extension == ".xlsx":
    xl = pd.ExcelFile(file_name)
    df = xl.parse("Sheet1", index_col=None, header=None)
if extension == ".txt":
    df = process_txt(file_name)

def process_txt(file_name):
    file = open(file_name, 'r', encoding="utf-8")
    lines = file.readlines()
    file.close()
    lines = [line[:-1] for line in lines]
    df = pd.DataFrame(data=lines)
    df.drop([0,1], inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.drop(range(3,len(df),4), inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.reset_index(inplace=True, drop=True)
    return df

df.columns = ['raw']

#drop number lines
df.drop(range(0,len(df),3), inplace=True)
df.reset_index(inplace=True, drop=True)

df_timestamp = df.iloc[0::2, :]
df_timestamp.reset_index(inplace=True, drop=True)

df_text = df.iloc[1::2, :]
df_text.reset_index(inplace=True, drop=True)

df = pd.concat([df_timestamp,df_text],axis=1)
df.columns = ['timestamp','text']


names = []
for index,row in df.iterrows():
    p = re.compile("^(\w+\s\w+):", re.I)
    name = p.search(row["text"])
    text = p.sub("",row["text"])
    df.loc[index]['text'] = text
    if name:
        names.append(name.group()[:-1])
    else:
        names.append("unclear")
    p2 = re.compile("\d\d:\d\d:\d\d.\d\d(?=\d)")
    inq_time = p2.search(row["timestamp"])
    df.loc[index]['timestamp'] = "["+inq_time.group()+"]"
df["names"]=names
df = df[["timestamp","names","text"]]
save_file_name = "cleaned_"+base_name+".xlsx"
df.to_excel(save_file_name, index=False, header=False)

timestamp            names  \
0    [00:00:03.36]  Margaret Foster   
1    [00:00:06.56]   Sarah Michaels   
2    [00:00:08.09]  Margaret Foster   
3    [00:00:13.13]   Sarah Michaels   
4    [00:00:22.14]   Sarah Michaels   
..             ...              ...   
232  [00:27:05.19]  Margaret Foster   
233  [00:27:23.52]  Margaret Foster   
234  [00:27:29.04]  Margaret Foster   
235  [00:27:33.00]   Sarah Michaels   
236  [00:27:37.50]   Sarah Michaels   

                                                  text  
0              Because if we record to the cloud, then  
1                             Record to your computer.  
2     No, because if we record to the cloud that th...  
3     Oh, well, we could try that I've just never d...  
4                                  Okay, let's try it.  
..                                                 ...  
232   And you click on the stripe and then it shows...  
233   Yeah, anyway, so there's there's a whole lot ...  
234                    Good at but that might be cool.  
235     So I'm going to suggest that we stop recording  
236   Because I don't think there's anything. What'...  

[237 rows x 3 columns]